<center>
<img src="https://compscicenter.ru/static/img/logo/yandexdataschool.e39870225d8b.svg" width=400px/>
<br />
<h1>Ускорение кода, Numpy, Numba, Cython, Pypy</h1>
<h3>Python</h3>
<br />
<h4>2017</h4> </center>

# `Matrix`

In [1]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

In [2]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

 

In [3]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))


[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


In [4]:
shape = (100, 100)

X = Matrix.random(shape)
Y = Matrix.random(shape)

In [5]:
%%timeit
matrix_product(X, Y)

421 ms ± 27.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
import cProfile
source = open("benchmark.py").read()
cProfile.run(source, sort="tottime") 

         100536 function calls in 4.299 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    4.235    0.424    4.236    0.424 <string>:26(matrix_product)
    20000    0.022    0.000    0.044    0.000 random.py:173(randrange)
    20000    0.015    0.000    0.022    0.000 random.py:223(_randbelow)
    20000    0.009    0.000    0.053    0.000 random.py:217(randint)
      200    0.008    0.000    0.061    0.000 <string>:13(<listcomp>)
    20038    0.005    0.000    0.005    0.000 {method 'getrandbits' of '_random.Random' objects}
    20000    0.002    0.000    0.002    0.000 {method 'bit_length' of 'int' objects}
        1    0.001    0.001    4.299    4.299 <string>:37(bench)
        1    0.001    0.001    4.299    4.299 {built-in method builtins.exec}
        2    0.000    0.000    0.061    0.031 <string>:9(random)
       10    0.000    0.000    0.000    0.000 <string>:7(<listcomp>)
        1    0.000    0.000    4.2

In [8]:
%load_ext line_profiler
def bench(shape=(100, 100), n_iter=10):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for iter in range(n_iter):
        matrix_product(X, Y)   

In [9]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z  

%lprun -f matrix_product bench()

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z


In [165]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [166]:
shape = (100, 100)

X = Matrix.random(shape)
Y = Matrix.random(shape)

In [36]:
%%timeit

matrix_product(X, Y)

280 ms ± 25.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%lprun -f matrix_product bench()

In [11]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            getX = Xi.__getitem__
            Zi[k] = sum(getX(j) * Ytk[j] for j in range(n_xcols))
    return Z

In [12]:
shape = (100, 100)
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [13]:
%%timeit
matrix_product(X, Y)

140 ms ± 8.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Numpy

In [14]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [15]:
shape = (100, 100)
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

In [16]:
%timeit np_matrix_product(X, Y)

525 ms ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit X.dot(Y)

572 µs ± 56.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Array

In [18]:
a = np.array([1.1, 2, 3])

a[1] = 5.1

print(a.size, a.nbytes, a.shape, a.dtype)
print(a)

3 24 (3,) float64
[ 1.1  5.1  3. ]


## UFunc

In [19]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i+j for (i, j) in zip(a, b)]

805 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
a, b = np.array(a), np.array(b)
%timeit a + b

19.2 ms ± 556 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

3.59 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [107]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

10.3 µs ± 718 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Aggregation

In [21]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

12.1 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
np_data = np.array(data)
%timeit np.min(np_data)

358 µs ± 4.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

%timeit np.dot(X, Y)

545 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Numba & Just-In-Time compilation

In [24]:
import numba
import numpy as np


@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [25]:
shape = 100, 100
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

In [27]:
%timeit -n 100 jit_matrix_product(X, Y)

728 µs ± 61.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Cython 

In [28]:
%load_ext Cython

In [29]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z


In [30]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [31]:
%timeit cy_matrix_product(X, Y)

81.1 ms ± 5.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [33]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [34]:
%timeit cy_matrix_product(X, Y)

446 ms ± 47.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit  cy_matrix_product(X, Y)

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np


cdef cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [39]:
%timeit -n10 cy_matrix_product(X, Y)

439 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
%%cython 
import numpy as np

cimport cython
cimport numpy as np
   
@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [53]:
%timeit cy_matrix_product(X, Y)

565 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Profit.